# Assignment 1. Student Performance Index
#### Dor Nevo 209498450
In this assignment, we will build models to predict student's performance. We will be using the "Student Performance Dataset" a dataset which includes student grades, demographic, social and school related features that has been collected by using school reports and questionnaires.
You can download the dataset from the moodle. You can also read on the dataset from [the UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/320/student+performance).

Our target value will be the student's performance in the end of the year which is marked 'G3' in the dataset. In the first part of the assignment you will be writing a linear regression model to predict the students grade at the end of the year. In the second part of the assignment you will build a classifier which predicts if the student's performance at the end of the year will be above average.

The assignment includes two files.
1. This file (ML_DL_Assignment1.ipynb)
2. A python utils file which you will fill out (ML_DL_Functions1.py)

As well as the student performance dataset file.

In this assignment you will mount and load the dataset and functions file from google drive. To start make sure you have both the template python functions file and the student dataset file on the same directory in your google drive.


**IMPORTANT:**
When you are finished with the assignment make sure you submit both ipynb file and the functions file. Take only the ipynb file and zip it to the name ML_DL_Assignment1.zip . Do not change the files names(ML_DL_Assignment1.zip, and ML_DL_Functions1.py) when submitting. This is crucial and can cause your code to fail in the testing.

In [2]:
import pandas
import numpy as np
import matplotlib.pyplot as plt
import sys


To make sure that you always have the newest version of your functions file(even if it changes whilst working on the assignment) we provided you with a reload function:

In [3]:
def reload_functions():
  if 'ML_DL_Functions1' in sys.modules:
    del sys.modules['ML_DL_Functions1']
  functions_path = drive_path.replace(" ","\ ") + 'ML_DL_Functions1.py'
  !cp $functions_path .

## 1. The Data



### 1.1 Load your dataset from the google drive
Start by setting up a Google Colab notebook in which to do your work.
you might find this link helpful:

- https://colab.research.google.com/github/googlecolab/colabtools/blob/master/notebooks/colab-github-demo.ipynb


To process and read the data, we use the popular `pandas` package for data analysis.

Now that your notebook is set up, we can load the data into the notebook. You will need to upload the "student-mat.csv" dataset and the functions file("ML_DL_Functions1.py") to a directory in google drive and mount your Google Drive to the colab notebook.

Here are some resources to help you get started:

[http://colab.research.google.com/notebooks/io.ipynb](https://colab.research.google.com/notebooks/io.ipynb)

In [4]:
from google.colab import drive
import sys
drive.mount('/content/gdrive')

drive_path = '/content/gdrive/My Drive/Intro_To_Deep_Learning/Assignment1/' # TODO - UPDATE ME WITH THE TRUE PATH!


print("looking for dataset in: "+drive_path+'student-mat.csv')
df = pandas.read_csv(drive_path+'student-mat.csv')
df = df.sample(frac=1) # This line scrambles the order of the dataframe

Mounted at /content/gdrive
looking for dataset in: /content/gdrive/My Drive/Intro_To_Deep_Learning/Assignment1/student-mat.csv


Now that the data is loaded to your Colab notebook, you should be able to display the Pandas
DataFrame `df` as a table:

In [5]:
df

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
36,GP,F,15,R,GT3,T,3,4,services,health,...,4,3,2,1,1,5,2,12,12,11
149,GP,F,16,U,GT3,T,4,2,services,other,...,4,2,3,1,1,5,2,15,16,16
57,GP,F,18,U,LE3,T,1,1,other,other,...,4,4,3,1,1,3,2,11,11,11
191,GP,F,17,U,GT3,A,4,4,other,teacher,...,4,1,4,1,1,1,6,6,5,6
92,GP,F,19,U,GT3,T,1,1,at_home,health,...,4,1,2,1,1,3,14,15,13,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,GP,F,16,U,GT3,T,1,3,at_home,services,...,4,3,5,1,1,3,0,8,7,0
293,GP,M,17,U,GT3,T,4,3,other,other,...,5,2,3,1,1,2,4,10,10,11
146,GP,M,17,U,GT3,T,2,1,other,other,...,4,5,1,1,1,3,2,8,8,10
255,GP,F,15,U,GT3,A,4,3,services,services,...,4,3,2,1,1,1,0,14,15,15


Notice that the data consists of multiple types. There are numeric columns such as 'age','freetime' and 'G3' but there are also a few categorical columns for example 'school','address','Mjob' etc. Since we want to use a numeric estimator we need all our data to be numeric. We can use indicators to translate the categories into numeric values. In the pandas library it is done using the [get_dummies](https://pandas.pydata.org/docs/reference/api/pandas.get_dummies.html) function.

Notice that in this assignment we will not be using any multiple category columns (to simplify the task a bit) so we remove the columns 'Mjob', 'Fjob', 'reason' and 'guardian'.

In [6]:
df_no_multi_categorical = df.drop(["Mjob","Fjob","reason","guardian"],axis=1)
df_numerical = pandas.get_dummies(df_no_multi_categorical, drop_first=True)

Notice how each categorical column has been converted to a binary numerical value.

In [7]:
df_numerical

,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,...,famsize_LE3,Pstatus_T,schoolsup_yes,famsup_yes,paid_yes,activities_yes,nursery_yes,higher_yes,internet_yes,romantic_yes
36,15,3,4,1,3,0,4,3,2,1,...,False,True,True,True,True,True,True,True,True,False
149,16,4,2,1,2,0,4,2,3,1,...,False,True,False,True,False,False,True,True,True,False
57,18,1,1,2,2,0,4,4,3,1,...,True,True,False,True,True,False,False,True,False,False
191,17,4,4,2,2,0,4,1,4,1,...,False,False,True,True,False,False,True,True,False,False
92,19,1,1,1,3,2,4,1,2,1,...,False,True,False,False,False,False,False,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,16,1,3,1,2,3,4,3,5,1,...,False,True,False,False,False,True,False,True,True,True
293,17,4,3,1,2,0,5,2,3,1,...,False,True,False,True,False,True,True,True,True,True
146,17,2,1,2,1,3,4,5,1,1,...,False,True,True,True,False,True,True,False,True,False
255,15,4,3,1,2,0,4,3,2,1,...,False,False,False,True,True,True,True,True,True,False


### 1.2 Prepare the data
Splitting a dataset into training and testing sets is a common practice in machine learning. The primary reason for doing so is to evaluate the performance of a model on unseen data. By splitting the dataset into training and testing sets, we can train our model on the training set and evaluate its performance on the testing set.

In this assignment the code used to prepare the data is already given to you


In [8]:
df_train = df_numerical[:270]
df_test = df_numerical[270:]

It can also be beneficial to **normalize** the columns, so that each column (feature)
has the *same* mean and standard deviation.

In [9]:
feature_means = df_train.mean().to_numpy()
feature_stds  = df_train.std().to_numpy()
feature_stds[feature_stds==0] = 0.01
train_norm = (df_train - feature_means) / feature_stds
test_norm = (df_test - feature_means) / feature_stds

#### Food for thought:
*Notice how in our code, we normalized the test* *set using the training data means and standard* *deviations. This is not a bug.Why would it would* *be improper to compute and use test set means and* *standard deviations. (Hint: Remember what we want* *to use the test accuracy to measure.)*



Finally we split the labels and the features of the dataset into seperate matrices.

In [11]:
# convert to numpy
train_x = train_norm.drop("G3",axis=1).to_numpy()
train_s = train_norm["G3"].to_numpy()
test_x = test_norm.drop("G3",axis=1).to_numpy()
test_s = test_norm["G3"].to_numpy()


### 1.3 check for correlation between the variables in the input
before you start fitting models to the data it is important to understand it. calculate and show (in whatever way fits you) the correlation between each of the input parameters('age','Medu','Fedu','traveltime', etc) and the predicted parameter 'G3')
you can use the [seaborn](https://seaborn.pydata.org/) library to aid you in this. This section is only for educational purposes, make sure you do not change the shape or values of the training or test data in it.

In [ ]:
import seaborn as sn
import matplotlib.pyplot as plt
# Insert your code  here

#### Food for thought:
*what can you learn from this test? are all input* *parameters equally important? which ones could you omit? which two variables are the best predictor of student performance at the end of the year and why?*

## 2. Linear regression
The first task you are going to tackle is Linear regression. In this task we will predict a students performance according to his different parameters.


### 2.1 Fit a linear model
 you can use the LinearRegression class from sklearn.linear_model. you can read more on it on:

 https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html


In [13]:
from sklearn.linear_model import LinearRegression

reg = LinearRegression()
reg.fit(train_x,train_s)
pred_test_s = reg.predict(test_x)

### 2.2 Test the accuracy of the model
There are several ways to measure the accuracy of a linear regression model. test your model using the '[predict](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)' function on the test set. you can measure the accuracy using MSE:

$MSE= \sum(s-\hat{s})^2$

or using the $R^2$ value:

$1-\frac{\text{unexplained variance=MSE}}{Total Variation}=1-\frac{\sum(s-\hat{s})^2}{\sum(s-\bar{s})^2}$

you can read on it further [here](https://en.wikipedia.org/wiki/Coefficient_of_determination).

In [53]:
sq_diff_test = (test_s - pred_test_s)**2
MSE_test = np.mean(sq_diff_test)
R_sq_test = 1 - ((sq_diff_test).sum()/((test_s - test_s.mean())** 2).sum())
print("MSE_test: "+str(MSE_test))
print("R_sq: "+str(R_sq_test))

MSE_test: 0.1433406155739004
R_sq: 0.860175374529663


If everything went correctly you should get an MSE value of around 0.11 and around 0.87 for $R^2$ .

### 2.3 save the parameters of the model

- Run the code block below.
- copy the coefficients list and paste it in the return value of the function 'linear_regression_coeff_submission' in the functions file **in your google drive**.
- copy the intercept value and paste it in the return value of the function 'linear_regression_intrcpt_submission' in the function file **in your google drive**.

THESE VALUES WILL BE USED TO GRADE YOU SO MAKE SURE YOU COPIED THEM CORRECTLY

In [15]:
coef = reg.coef_
intrcpt = reg.intercept_
print("coefficients: ")
print(coef)
print("intercept: ")
print(intrcpt)

coefficients: 
[-7.12910261e-02 -4.73665169e-03 -1.03978610e-02  8.65626404e-03
 -3.99926260e-02  3.40262672e-03  6.61865174e-02  7.03472933e-05
  2.03956248e-02 -3.19123317e-02  4.35763050e-02  8.35492096e-03
  8.40613913e-02  1.80697557e-01  7.60750981e-01  3.21130109e-02
  1.44608543e-02 -1.02015057e-02  1.96121548e-02 -5.94568044e-03
  3.47496714e-02  3.60067713e-02  2.90424729e-03 -3.22749177e-02
 -3.56838685e-02  1.20871461e-02 -5.53989290e-03 -2.56530532e-02]
intercept: 
1.8989779321778998e-18


To make sure you copied the values correctly you can run the following block:

In [16]:
reload_functions()
import ML_DL_Functions1
if abs(ML_DL_Functions1.linear_regression_coeff_submission() - coef).sum()>(10**(-6)):
  print("Coefficients were NOT copied correctly")
elif ML_DL_Functions1.linear_regression_intrcpt_submission() != intrcpt:
  print("Intercept Value was NOT copied correctly")


### 2.4 test your skills in Linear algebra and compare the result to sklearn
While using Sklearn's built in linear regression is easy it is also important to see that it is not too complicated to write the regression by yourself. In class you were tought that the linear regression could be estimated using the Least Squares method. The Psudo-Inverse is defined as
$\theta^* = (X^TX)^{-1}X^Ts$. Use this knowledge to write the linear regression yourself. Fill in the LeastSquares function in the functions file.
In this function you are not allowed to use the sklearn or scipy libraries.

In [54]:
import scipy

from copy import deepcopy
reload_functions()
import ML_DL_Functions1
train_x_tag = np.hstack([np.ones([train_x.shape[0],1]),train_x])# we add the bias term directly to the linear equation
theta = ML_DL_Functions1.LeastSquares(train_x_tag,train_s)



Test the accuracy of the model, as in 2.2

In [55]:
test_x_tag = np.hstack([np.ones([test_x.shape[0],1]),test_x])
pred_test_s_tag = test_x_tag @ theta
sq_diff_test_tag = (test_s - pred_test_s_tag)**2
MSE_test_tag = np.mean(sq_diff_test_tag)
R_sq_test_tag = 1 - (sq_diff_test_tag.sum()/((test_s - test_s.mean())** 2).sum())
print("MSE_test_tag: "+str(MSE_test_tag))
print("R_sq_tag: "+str(R_sq_test_tag))




MSE_test_tag: 0.14334061557390038
R_sq_tag: 0.860175374529663


## 3. Linear classification
In this section you will be classifying wether a students performance in the end of the year was above average or not according to all other metrics.

### 3.1 prepare the data: split the dataset between students with grades above average and below and then split again for train test

To set up our data for classification, we'll use the "G3" field to represent whether a student achieved a performance above average or below. In our case `df_cl["G3"]` will be 1 if the students performance is above average and 0 otherwise.

In [29]:
from copy import deepcopy
df_cl = deepcopy(df_numerical)
print("average performance= "+ str(df_cl['G3'].mean()))
df_cl["G3"] = df["G3"].map(lambda x: int(x > 10))

average performance= 10.494117647058824


Next you need to normalize and split the dataframe into a training and test sets just like was done in part 1.2.

**Notice that when normalizing you do not normalize the labels**

In [65]:
df_cl_train = df_cl[:270]
df_cl_test = df_cl[270:]

feature_cl_means = df_cl_train.mean().to_numpy()
feature_cl_stds  = df_cl_train.std().to_numpy()
feature_cl_stds[feature_cl_stds==0] = 0.01
train_cl_norm = (df_cl_train - feature_cl_means) / feature_cl_stds
test_cl_norm = (df_cl_test - feature_cl_means) / feature_cl_stds

train_cl_x = train_cl_norm.drop("G3",axis=1).to_numpy()
train_cl_s = df_cl_train["G3"].to_numpy()
test_cl_x = test_cl_norm.drop("G3",axis=1).to_numpy()
test_cl_s = df_cl_test["G3"].to_numpy()



### 3.2 Fit an svm model to the data to predict whether a grade is above or below average
Read on [the svm implementation of sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html). fit the model on the training classification dataset you built.

In [70]:
import sklearn.svm
svc = sklearn.svm.LinearSVC()

svc.fit(train_cl_x, train_cl_s)


LinearSVC()

### 3.3 Test your model
For classification accuracy is usually calculated as the number of correct predictions divided by number of total predictions:
$$\frac{\text{Number of correct predicitions}}{\text{Number of total predictions}}$$

The accuracy is calculated on the test set. Fill in the accuracy function in the functions file and run the test below:

In [71]:
reload_functions()
import ML_DL_Functions1
ML_DL_Functions1.classification_accuracy(svc,test_cl_x,test_cl_s)

0.8571428571428571

If everything worked correctly you should get an accuracy of around  90%

### 3.4 save the parameters of the model
- Run the code block below.
- copy the coefficients list and paste it in the return value of the function 'classification_coeff_submission' in the functions file **in your google drive**.
- copy the intercept value and paste it in the return value of the function 'classification_intrcpt_submission' in the functions file **in your google drive**.
- copy the classes value and paste it in the return value of the function 'classification_classes_submission' in the functions file **in your google drive**.

THESE VALUES WILL BE USED TO GRADE YOU SO MAKE SURE YOU COPIED THEM CORRECTLY

In [73]:
coef_cl = svc.coef_
intrcpt_cl = svc.intercept_
classes_cl = svc.classes_
print(coef_cl)
print(intrcpt_cl)
print(classes_cl)

[[-0.25220493 -0.29741098  0.16431881 -0.14486237 -0.13429201 -0.32733827
  -0.22000506  0.0811542  -0.11061615  0.17684061 -0.30864931  0.04196098
  -0.10956331  1.08033923  2.86402312 -0.4430299  -0.09542346 -0.0650778
  -0.19082981 -0.19187785  0.22886821  0.25540807  0.04251017 -0.11308635
  -0.46784574  0.09329152  0.0533493   0.39909522]]
[0.29508435]
[0 1]


Make sure you copied your values correctly:

In [74]:
reload_functions()
import ML_DL_Functions1
if abs(ML_DL_Functions1.classification_coeff_submission() - coef_cl).sum()>(10**(-6)):
  print("Coefficients were NOT copied correctly")
elif ML_DL_Functions1.classification_intrcpt_submission()[0]- intrcpt_cl[0]>(10**(-6)):
  print("Intercept Value was NOT copied correctly")
elif (ML_DL_Functions1.classification_classes_submission() - classes_cl).sum()!=0:
  print("Intercept Value was NOT copied correctly")
